<a href="https://colab.research.google.com/github/wajnryt/JerusMLDeepLearning2019/blob/David/Resnet18IntstanceClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode


from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import glob
basedir = '/content/gdrive/My Drive/videos_2/yt_bb_detection_train'

In [0]:
import pandas as pd
import os
from torch.utils.data.dataset import Dataset
import imageio
from PIL import Image

class InstanceDataset(Dataset):
    def __init__(self, basedir, transform=None, force=False):
        super().__init__()
        self.transform = transform
        cach_file = os.path.join(basedir, 'data.csv')
        if not force and os.path.exists(cach_file):
          self.data = pd.read_csv(cach_file)
          print('load from csv')
          return
        print('start glob')
        files = glob.glob(os.path.join(basedir ,'*','*','*.jpg'))
        print('finish glob')
        self.data = pd.DataFrame([self._split_file(f) for f in files], 
                            columns=['class_id',  'file_path'])
        self.data.to_csv(cach_file)
        print('finish sort')
        
    def _split_file(self, f):
        parts = f.split(os.sep)[-3:-1]
        return parts[0], f 

    def __getitem__(self, index):
      dat = self.data.iloc[index]
      img = Image.open(dat['file_path'])
      if self.transform:
        img = self.transform(img)
      return (img, dat['class_id'], dat['instance_id'])
              
    def __len__(self):
        return len(self.data)
      

In [0]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256,256)),
        #transforms.Pad(256),
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# dataset = InstanceDataset(basedir, data_transforms['train'], True)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=3,
#                                              shuffle=True, num_workers=4)

device = torch.device("cuda:0")


In [0]:

device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet18(pretrained=True)


optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
# train_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/Grayscale/', transform=ToTensor())
# test_set = ImageFolder(root='/content/JerusML/My Drive/JerusML/GrayscaleTEST/', transform=ToTensor())

batch_size = 3
dataset_train = InstanceDataset(basedir, data_transforms['train'], True)
dataset_val = InstanceDataset(basedir, data_transforms['val'], True)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                             shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size,
                                             shuffle=True, num_workers=4)



# train_loader = torch.utils.data.DataLoader(
#                  dataset=train_set,
#                  batch_size=batch_size,
#                  shuffle=True)
# test_loader = torch.utils.data.DataLoader(
#                 dataset=test_set,
#                 batch_size=batch_size,
#                 shuffle=False)


print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
print ('==>>> total testing batch number: {}'.format(len(test_loader)))

## network
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1, 2)
        self.conv2 = nn.Conv2d(20, 50, 5, 1, 2)
        self.fc1 = nn.Linear(164*218*50, 3)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        print(x.shape)
        x = x.view(-1, 164*218*50)
        x = self.fc1(x)
        return x
    
    def name(self):
        return "LeNet"

## training
model = models.resnet18()
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    # trainning
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss,float(correct_cnt)/total_cnt))

torch.save(model.state_dict(), model.name())


start glob
finish glob


In [0]:
# print(len(dataset.data['instance_id'].unique()), ' unique instances')
# print(len(dataset.data['class_id'].unique()), ' classes')
# print(len(dataset) , ' images')
# import numpy as np

# plt.figure(figsize=(15,15))
# plt.hist(dataset.data['class_id'].values.astype(int), normed=True)
# plt.title('images per class')
# plt.xlabel('class id')

# plt.figure(figsize=(15,15))
# _, count = np.unique(dataset.data['instance_id'], return_counts=True)
# plt.hist(count, normed=True)
# plt.title('images per instance')
# plt.xlabel('images per instance')

In [0]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# #example. 
# # this is not how you're supposed to use dataset!
# # you should axes it via torch DataLoader...
# # never dirrectly call __getitem__, or access the members (dataset.data)

# print(len(dataset))
# im, class_id, instance_id = dataset.__getitem__(500)


# #just for fun, lets find another one of this guy
# thisguy = dataset.data[dataset.data['instance_id']==instance_id]
# for ind, dat in thisguy.iterrows():
#   plt.figure()
#   plt.imshow(imageio.imread(dat['file_path']))
#   plt.title(f'class id {dat["class_id"]}, instance {dat["instance_id"]}')


In [0]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes, instance= next(iter(dataloader))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs,3)
# plt.figure(figsize=(15,15))
# imshow(out)#, title=[class_names[x] for x in classes])

In [116]:
# import torch
# import torch.nn as nn
# from torch.autograd import Variable
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision.datasets import ImageFolder
# from torchvision.transforms import ToTensor


# batch_size = 3
# dataset_train = InstanceDataset(basedir,dataset_train, True)
# dataset_val = InstanceDataset(basedir, dataset_train, True)

# train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
#                                              shuffle=True, num_workers=4)

# test_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size,
#                                              shuffle=True, num_workers=4)

# print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
# print ('==>>> total testing batch number: {}'.format(len(test_loader)))


# model = models.resnet18()

# epochs = 1
# steps = 0
# running_loss = 0
# print_every = 10
# train_losses, test_losses = [], []
# for epoch in range(epochs):
#     for inputs, labels in trainloader:
#         steps += 1
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         logps = model.forward(inputs)
#         loss = criterion(logps, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
        
#         if steps % print_every == 0:
#             test_loss = 0
#             accuracy = 0
#             model.eval()
#             with torch.no_grad():
#                 for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device), labels.to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals =  top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(trainloader))
#             test_losses.append(test_loss/len(testloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(testloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()
# torch.save(model, 'resnet18i.pth')

start glob
finish glob
finish sort
start glob
finish glob
finish sort
==>>> total trainning batch number: 33004
==>>> total testing batch number: 33004


RuntimeError: ignored